# Data Processing

In [11]:
import pandas as pd
from googlesearch import search
import wordninja
import requests
from bs4 import BeautifulSoup

In [12]:
data = pd.read_csv('testdata_Labeled.csv')

In [13]:
data_df = data.head(73)

In [14]:
data_df.dropna(how='all',axis=1,inplace=True)

/home/roshan/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
data_df.dropna(how='all',axis=0,inplace=True)

/home/roshan/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [16]:
data_df.drop(axis=1,labels=[data_df.columns[3],data_df.columns[4]],inplace=True)

/home/roshan/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
data_df.drop(data_df.index[73],inplace = True)

IndexError: index 73 is out of bounds for axis 0 with size 72

In [18]:
data_df.tail()

,housing_flg,Google’s First Url,Category
68,ALLAHABAD BANK,https://www.allahabadbank.in/,Bank
69,ALLAHABADBANKLNRP,https://www.allahabadbank.in/,Bank
70,ALLAHABADBANKUTBP,https://www.sulekha.com/itbp-exam-coaching/all...,Bank
71,AMERICAN EXPRESS BAN,https://www.americanexpress.com/in/,Bank
72,ANAND RATHI GLOBAL F,https://www.rathi.com/,Finance


In [19]:
from sklearn.cross_validation import train_test_split
X = data_df.housing_flg
y = data_df.Category
X_train,X_test,Y_train,Y_test = train_test_split(X,y,random_state = 1)

/home/roshan/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Transforming the train and test data

In [20]:
def transform_data (inputseries):
    X_df = pd.DataFrame(inputseries)
    X_df['Company'] = 'Nan'
    for i in range(5):
        stri = 'Link'+str(i)
        X_df[stri] = 'Nan'
        stri = 'Data'+str(i)
        X_df[stri] = 'Nan'
    return X_df

In [ ]:
X_df = transform_data(X)

## Obtaining LINK

In [42]:
def link_obtain(X_df):
    for i,j in X_df.iterrows():
        URL = X_df['housing_flg'][i]
        query = URL
        print(query)
        #Directly Retrieving all ifsc code data without saving link in column.
        if(len(str(query.strip())))<5:
            t =0
            str_list = list('https://www.ifsccodebank.com/search-by-IFSC-code.aspx?IFSCCode=')
            str_list.append(query)
            url = ''.join(str_list)
            response = requests.get(''.join(str_list))
            soup = BeautifulSoup(response.text,'html.parser')
            t = soup.get_text()
            r = t[t.find(query)+7:]
            r = (r[:r.find('-')-5].strip())
            X_df['Company'][i] = r
            q2 = str(X_df['Company'][i])
            print(q2)
            if 'com' not in q2:
                t =0
                for l in search(q2,stop = 2):
                    stri = 'Link'+str(t)
                    print(q2,l)
                    X_df[stri][i] = l
                    t = int(t)+1
                    print(stri)
            else:
                X_df['Company'][i] = 'Nan'
            continue
        #if Name has no space ---> Using wordninja algo to split the words.
        elif ' ' not in query.strip():
            query = " ".join(str(x) for x in wordninja.split(query))
    ####===> The following gets the link and not the data <===####    
        # Direct search for appropriate official Link    
        try:
            t = 0
            for k in search(query,stop = 2):
                stri = 'Link' + str(t)
                X_df[stri][i] = k
                t = t+1
                print(stri)
        # Search the words with key word Financial Services
            t = 2
            query = URL + ' Finanancial Services'
            for k in search(query,stop = 2):
                stri = 'Link' + str(t)
                X_df[stri][i] = k 
                t = t+1
                print(stri)
        except Exception as e:
            print(e)
            # except block to store the company which has bad request/other issue
            X_df['Link4'][i] = query
            print('Errored out: ',query)
    return X_df

In [ ]:
X_df = link_obtain(X_df)

## Obtaining Data

In [43]:
def cleanme(html):
    soup = BeautifulSoup(html) # create a new bs4 object from the html data loaded
    for script in soup(["script"]): 
        script.extract()
    text = soup.get_text()
    text = text.replace('\n', ' ').replace('\r', '').replace('\t',' ')
    return text.strip()
    
def lmtd_data_desc(X_df):
    for i,j in X_df.iterrows():
        if 'Nan' not in X_df['Link0'][i]:
            response = requests.get(X_df['Link0'][i],verify = False)
            t = cleanme(response.text)
            X_df['Data0'][i] = t
    return X_df
def data_desc(X_df):
    for i,j in X_df.iterrows():
        try:
            if 'Nan' not in X_df['Link3'][i]:
                response = requests.get(X_df['Link3'][i],verify = False)
                t = cleanme(response.text)
                X_df['Data3'][i] = t
            if 'Nan' not in X_df['Link0'][i]:
                response = requests.get(X_df['Link0'][i],verify = False)
                t = cleanme(response.text)
                X_df['Data0'][i] = t
        except:
            print('Error: ',i)
    return X_df

In [ ]:
X_df = data_desc(X_df)

In [ ]:
X_df

In [ ]:
X_df.to_csv('LabeledData73.csv')

# Checking for accuracy after cleaning data\

In [1]:
def data_clean(content):
    # 1. SPLITTING INTO WORDS
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(str(content))
    # 2. CONVERT WORDS TO LOWERCASE
    words_punc=list()
    for i in tokens:
        words_punc.append(i.lower())
    # 3. REMOVE PUNCTUATIONS
    import string
    table = str.maketrans('','',string.punctuation)
    alphanum = [i.translate(table) for i in words_punc ]
    # 4. REMOVE NOT CHAR TOKENS
    alphaonly = [i for i in alphanum if i.isalpha() ]
    # 5. REMOVE STOPWORDS
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words = [i for i in alphaonly if not i in stop_words]

    ### Lemmatizing OF WORDS
    #!!!!! ---- NOT STEMMING BECAUSE IT CUTS SHORT THE IMPORTANT WORDS -----!!!!#
    # NEED TO TRY WITH STEMMING #

    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    word = list()
    for i in words:
        word.append(lemmatizer.lemmatize(i,'v'))

    return(word)

# Vectorizing using TFIDF vectorizer

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf = 'False',binary=True,ngram_range=(1,2))
vectorizer.fit(['Finance','Fund','Mutual Fund', 'Loan', 'Insurance', 'Investor' ,'Business Services','Financial Services', 'Personal Loan', 'Mobile Loan', 'Consumer Durable Loan', 'Furniture Loan', '2-wheeler Loan', 'E - Bike Loan', 'Bank', 'Home Loan', 'Housing Finance'])
vectorizer.get_feature_names()

['bank',
 'bike',
 'bike loan',
 'business',
 'business services',
 'consumer',
 'consumer durable',
 'durable',
 'durable loan',
 'finance',
 'financial',
 'financial services',
 'fund',
 'furniture',
 'furniture loan',
 'home',
 'home loan',
 'housing',
 'housing finance',
 'insurance',
 'investor',
 'loan',
 'mobile',
 'mobile loan',
 'mutual',
 'mutual fund',
 'personal',
 'personal loan',
 'services',
 'wheeler',
 'wheeler loan']

### Vectorization Transformation

In [3]:
def transform_to_vector(vectorizer,X_df):
    X_res = pd.DataFrame(columns = vectorizer.get_feature_names())
    for i,j in X_df.iterrows():
        try:
            print(X_df['housing_flg'][i])
            rtlst = [0]*len(vectorizer.get_feature_names())
            rt = vectorizer.transform([X_df['Data0'][i]]).toarray()
            for k in range(rt.shape[1]):
                rtlst[k] = rt[0][k]+rtlst[k]
                print ('Initial ',i, rtlst[k])
            rt = vectorizer.transform([X_df['Data3'][i]]).toarray()
            for k in range(rt.shape[1]):
                rtlst[k] = rt[0][k]+rtlst[k]
                print('Final ',i,rtlst[k])
            X_res.loc[i] = rtlst
        except:
            for k in range(rt.shape[1]):
                rtlst[k] = 0
                print('Errored out ',X_df['housing_flg'][i],rtlst[k])
            X_res.loc[i] = rtlst
    return X_res

In [21]:
X_df = pd.read_csv('LabeledLimitedData73.csv')
X_df.drop(X_df.columns[0],axis = 1,inplace = True)
X_df.head()

,housing_flg,Company,Link0,Data0,Link1,Data1,Link2,Data2,Link3,Data3,Link4,Data4
0,AADHAR HOUSING FINAN,Nan,https://aadharhousing.com/,Aadhar Housing Finance Ltd: Home Loan | House ...,https://aadharhousing.com/ready-reckoner/home-...,Nan,https://aadharhousing.com/investor-relations/a...,Nan,https://aadharhousing.com/,Nan,Nan,Nan
1,AAMAADMIPARTY,Nan,https://www.youtube.com/watch?v=iGm8ks1rEdE,@font-face{font-family:'Roboto';font-style:ita...,https://www.facebook.com/AamAadmiParty/videos/...,Nan,https://eci.gov.in/files/file/9217-aam-aadmi-p...,Nan,https://en.wikipedia.org/wiki/Aam_Aadmi_Party,Nan,Nan,Nan
2,Aavas Financiers Lim,Nan,https://www.aavas.in/,"Aavas Financiers Limited - Home Loan, Home Loa...",https://www.aavas.in/about-us,Nan,https://www.aavas.in/about-us,Nan,https://www.aavas.in/meet-my-team,Nan,Nan,Nan
3,ABHIPRA CAPITAL LTD,Nan,https://www.abhipra.com/,.async-hide { opacity: 0 !important} Ab...,https://www.abhipra.com/contact-us,Nan,https://www.abhipra.com/,Nan,https://www.abhipra.com/company-info/contact-us,Nan,Nan,Nan
4,ANDB,andhra bank,https://www.andhrabank.in/,www.andhrabank.in,https://www.andhrabank.in/netbanking/index-ret...,Nan,Nan,Nan,Nan,Nan,Nan,Nan


In [10]:
X_res = transform_to_vector(vectorizer,X_df)
X_res

AADHAR HOUSING FINAN
Initial  0 0.28207893665052103
Initial  0 0.0
Initial  0 0.0
Initial  0 0.28207893665052103
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.24630629579357813
Initial  0 0.28207893665052103
Initial  0 0.28207893665052103
Initial  0 0.24630629579357813
Initial  0 0.0
Initial  0 0.0
Initial  0 0.28207893665052103
Initial  0 0.28207893665052103
Initial  0 0.28207893665052103
Initial  0 0.28207893665052103
Initial  0 0.28207893665052103
Initial  0 0.28207893665052103
Initial  0 0.14937992147023105
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.24630629579357813
Initial  0 0.0
Initial  0 0.0
Final  0 0.28207893665052103
Final  0 0.0
Final  0 0.0
Final  0 0.28207893665052103
Final  0 0.0
Final  0 0.0
Final  0 0.0
Final  0 0.0
Final  0 0.0
Final  0 0.24630629579357813
Final  0 0.28207893665052103
Final  0 0.28207893665052103
Final  0 0.24630629579357813
Final  0 0.0
Final  0 0.0

/home/roshan/anaconda3/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Initial  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
Final  41 0.0
ACHDRTPZ DT 
Initial  42 0.0
Initial  42 0.0
Initial  42 0.0
Initial  

,bank,bike,bike loan,business,business services,consumer,consumer durable,durable,durable loan,finance,...,loan,mobile,mobile loan,mutual,mutual fund,personal,personal loan,services,wheeler,wheeler loan
0,0.282079,0.00000,0.00000,0.282079,0.0,0.000000,0.00000,0.00000,0.00000,0.246306,...,0.149380,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.246306,0.000000,0.000000
1,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.00000,0.00000,0.00000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.00000,0.00000,0.00000,0.434269,...,0.263376,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.324019,0.00000,0.00000,0.324019,0.0,0.000000,0.00000,0.00000,0.00000,0.000000,...,0.000000,0.324019,0.00000,0.324019,0.324019,0.000000,0.000000,0.282927,0.000000,0.000000
4,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.00000,0.00000,0.00000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.238584,0.00000,0.00000,0.238584,0.0,0.000000,0.00000,0.00000,0.00000,0.208327,...,0.126346,0.238584,0.00000,0.238584,0.238584,0.238584,0.238584,0.208327,0.238584,0.000000
6,0.253447,0.00000,0.00000,0.253447,0.0,0.000000,0.00000,0.00000,0.00000,0.221305,...,0.134217,0.253447,0.00000,0.253447,0.253447,0.253447,0.253447,0.221305,0.000000,0.000000
7,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.00000,0.00000,0.00000,0.000000,...,0.215427,0.406797,0.00000,0.000000,0.000000,0.406797,0.406797,0.355208,0.000000,0.000000
8,0.304215,0.00000,0.00000,0.304215,0.0,0.000000,0.00000,0.00000,0.00000,0.000000,...,0.161103,0.304215,0.00000,0.304215,0.000000,0.304215,0.000000,0.265635,0.000000,0.000000
9,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.00000,0.00000,0.00000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
X_res.to_csv('X_final.csv')
pd.DataFrame(y).to_csv('Y_final.csv')

#storing
pd.DataFrame(Y_train).to_csv('Y_Res2.csv')

pd.DataFrame(Y_test).to_csv('Y_Res3.csv')

In [ ]:
Y_train = pd.read_csv('Y_Res2.csv')
Y_test = pd.read_csv('Y_Res3.csv')
y = pd.concat([Y_train,Y_test])

# With limited Data description:

import pandas as pd
X_train = pd.read_csv('X_Res2.csv')
X = pd.concat([X_train,pd.read_csv('X_Res3.csv')])

In [200]:
import pandas as pd
X = pd.read_csv('X_final.csv')

In [22]:
y = pd.read_csv('Y_final.csv')

In [202]:
X.drop(X.columns[0],axis = 1,inplace = True)

In [23]:
y.drop(y.columns[0],axis = 1,inplace = True)

X_test = pd.read_csv('X_Res3.csv')

X.drop(X.columns[0],axis = 1,inplace=True)

## Using Multinomial NB

X_test.rename(columns = {X_test.columns[0]:'Index'},inplace=True)
X_train.rename(columns = {X_train.columns[0]:'Index'},inplace=True)
Y_test.rename(columns = {Y_test.columns[0]:'Index'},inplace=True)
Y_train.rename(columns = {Y_train.columns[0]:'Index'},inplace=True)

### With less data accuracy of Multinomial NB => approx 37%. 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
categories = ['Finance','Assmt','Investor','Loan','Bank','Home Loan','other']
model = MultinomialNB(alpha = 1)
cross_val_score(model,X,y.Category,cv = 30)
# for k in range(5):
# model.fit(X_train.drop(X_train.columns[0],axis = 1),Y_train.drop(Y_train.columns[0],axis = 1))
# labels = model.predict(X_test.drop(X_test.columns[0],axis = 1))
# # print('score for alpha ', k,' is ' , accuracy_score(labels,Y_test.drop(Y_test.columns[0],axis = 1)))
# confusion_matrix(Y_test.drop(Y_test.columns[0],axis = 1),labels)

## Using KNN neighbour


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
modelknn = KNeighborsClassifier(n_neighbors=5)
cross_val_score(modelknn,X,y.Category,cv = 4)
# for k in range(25):
#     modelknn = KNeighborsClassifier(n_neighbors=(k+1))
#     modelknn.fit(X_train.drop(X_train.columns[0],axis = 1),Y_train.drop(Y_train.columns[0],axis = 1))
#     labelknn = modelknn.predict(X_test.drop(X_test.columns[0],axis =1))
#     scr = accuracy_score(Y_test.drop(Y_test.columns[0],axis = 1),labelknn)
#     print('Accuracy for ',k+1,' neighbors = ',scr)

# With more Data description:=>

import pandas as pd
X_train = pd.read_csv('X_fin1.csv')

X_test = pd.read_csv('X_fin2.csv')

X = pd.concat([X_train,X_test])
X.drop(X.columns[0],axis =1 )

## Using Multinomial NB

In [ ]:
X_test.rename(columns = {X_test.columns[0]:'Index'},inplace=True)
X_train.rename(columns = {X_train.columns[0]:'Index'},inplace=True)
Y_test.rename(columns = {Y_test.columns[0]:'Index'},inplace=True)
Y_train.rename(columns = {Y_train.columns[0]:'Index'},inplace=True)

### With more data accuracy of Multinomial NB falls drastically. => avg approx = 26% 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
model = MultinomialNB(alpha = 1)
categories = ['Finance','Assmt','Investor','Loan','Bank','Home Loan','other']
for k in range(5):
    model = MultinomialNB(alpha = k)
    model.fit(X_train.drop(X_train.columns[0],axis = 1),Y_train.drop(Y_train.columns[0],axis = 1))
    labels = model.predict(X_test.drop(X_test.columns[0],axis = 1))
    print('score for alpha ', k,' is ' , accuracy_score(labels,Y_test.drop(Y_test.columns[0],axis = 1)))

## Using KNN neighbour


In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
modelknn = KNeighborsClassifier(n_neighbors=5)
for k in range(25):
    modelknn = KNeighborsClassifier(n_neighbors=(k+1))
    modelknn.fit(X_train,Y_train)
    labelknn = modelknn.predict(X_test)
    scr = accuracy_score(Y_test,labelknn)
    print('Accuracy for ',k+1,' neighbors = ',scr)

Accuracy for  1  neighbors =  0.7142857142857143
Accuracy for  2  neighbors =  0.5714285714285714
Accuracy for  3  neighbors =  0.5714285714285714
Accuracy for  4  neighbors =  0.5714285714285714
Accuracy for  5  neighbors =  0.5714285714285714
Accuracy for  6  neighbors =  0.5714285714285714
Accuracy for  7  neighbors =  0.5714285714285714
Accuracy for  8  neighbors =  0.5714285714285714
Accuracy for  9  neighbors =  0.5714285714285714
Accuracy for  10  neighbors =  0.5714285714285714
Accuracy for  11  neighbors =  0.7142857142857143
Accuracy for  12  neighbors =  0.7142857142857143
Accuracy for  13  neighbors =  0.7142857142857143
Accuracy for  14  neighbors =  0.7142857142857143
Accuracy for  15  neighbors =  0.7142857142857143
Accuracy for  16  neighbors =  0.8571428571428571
Accuracy for  17  neighbors =  0.8571428571428571
Accuracy for  18  neighbors =  0.8571428571428571
Accuracy for  19  neighbors =  0.8571428571428571
Accuracy for  20  neighbors =  0.8571428571428571
Accuracy 

In [101]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [106]:
bf = SelectKBest(score_func=chi2)

In [107]:
fit = bf.fit(X_train,Y_train)

In [108]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_train.columns)
fscore = pd.concat([dfscores,dfcolumns],axis = 1)
fscore.columns = ['Score','Specs']
fscore.nlargest(30,'Score')

,Score,Specs
1,13.807125,bike
2,13.807125,bike loan
6,13.807125,consumer durable
7,13.807125,durable
8,13.807125,durable loan
13,13.807125,furniture
14,13.807125,furniture loan
23,13.807125,mobile loan
11,8.243879,financial services
18,6.150780,housing finance


# Validation:= >

## Stratified K fold Validation

In [207]:
r,c = y.shape
y = y.values.reshape(r,)
y.shape

(73,)

## Getting Feature importance by dropping each column and getting accuracy -->>

In [264]:
val = [0]*X.shape[1]

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=7,random_state=None)
for trainind,testind in skf.split(X,y):
    X_train,X_test = X.loc[trainind],X.loc[testind]
    Y_train,Y_test = y[trainind],y[testind]
    modelknn.fit(X_train,Y_train)
    labeled = modelknn.predict(X_test)
    init = accuracy_score(labeled,Y_test)
    finallst = list()
    count = 0
    for i in X_train.columns:
        Xmiss = X_train
        Xmiss = X_train.drop(i,axis = 1)
        modelknn.fit(Xmiss,Y_train)
        finallst.append(accuracy_score(modelknn.predict(X_test.drop(i,axis = 1)),Y_test))
        val[count] = val[count]+finallst[count]
        count = count+1
    print(init)
count = 0
ls = list()
for i in X.columns:
    ls.append((i,val[count]/4))
    count = count+1
pd.DataFrame(ls,columns = {'Feature','Mean Accuracy after drop'})

/home/roshan/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=7.
  % (min_groups, self.n_splits)), Warning)


0.4
0.46153846153846156
0.3333333333333333
0.5
0.6666666666666666
0.7142857142857143
0.8571428571428571


,Feature,Mean Accuracy after drop
0,bank,0.746368
1,bike,1.004075
2,bike loan,1.004075
3,business,1.045742
4,business services,0.983242
5,consumer,1.004075
6,consumer durable,1.004075
7,durable,1.004075
8,durable loan,1.004075
9,finance,1.004075


## <-- Drop and check accuracy

In [100]:
from sklearn.metrics import confusion_matrix
modelknn = KNeighborsClassifier(n_neighbors=(18))
modelknn.fit(X_train,Y_train)
labelknn = modelknn.predict(X_test)
cm = confusion_matrix(Y_test,labelknn)
accuracy_score(Y_test,labelknn)
labelknn,Y_test

(array(['Bank', 'other', 'Bank', 'Bank', 'Bank', 'Bank', 'Bank', 'other',
        'Bank', 'Bank', 'Bank', 'Bank', 'Bank', 'Bank', 'Bank'],
       dtype=object),
 array(['Bank', 'Bank', 'Bank', 'other', 'other', 'other', 'Home Loan',
        'other', 'Investor', 'Investor', 'Bank', 'Bank', 'Bank', 'Bank',
        'Finance'], dtype=object))

0.4666666666666667


,Dropped,Accuracy
0,bank,0.400000
1,bike,0.466667
2,bike loan,0.466667
3,business,0.733333
4,business services,0.466667
5,consumer,0.666667
6,consumer durable,0.466667
7,durable,0.466667
8,durable loan,0.466667
9,finance,0.666667


# Testing with unknown data

In [60]:
comp_name = 'housing finance'

In [61]:
comp_name_x = pd.DataFrame([comp_name],columns = {'housing_flg'})

In [62]:
comp_name_x = transform_data(comp_name_x)

In [63]:
comp_name_x = link_obtain(comp_name_x)

housing finance
Link0
Link1
Link2
Link3


In [64]:
data_comp = lmtd_data_desc(comp_name_x)
data_comp

/home/roshan/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,housing_flg,Company,Link0,Data0,Link1,Data1,Link2,Data2,Link3,Data3,Link4,Data4
0,housing finance,Nan,https://www.dhfl.com/,Home Loan - Housing loan - Housing Finance com...,https://www.pnbhousing.com/,Nan,https://lichousing.com/subsidiary1.php,Nan,http://www.lichousing.com/,Nan,Nan,Nan


In [65]:
X_res = transform_to_vector(vectorizer,data_comp)
X_res

housing finance
Initial  0 0.31555180273416067
Initial  0 0.0
Initial  0 0.0
Initial  0 0.31555180273416067
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.27553420537291096
Initial  0 0.31555180273416067
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.31555180273416067
Initial  0 0.31555180273416067
Initial  0 0.31555180273416067
Initial  0 0.31555180273416067
Initial  0 0.31555180273416067
Initial  0 0.0
Initial  0 0.16710607346984868
Initial  0 0.31555180273416067
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Final  0 0.31555180273416067
Final  0 0.0
Final  0 0.0
Final  0 0.31555180273416067
Final  0 0.0
Final  0 0.0
Final  0 0.0
Final  0 0.0
Final  0 0.0
Final  0 0.27553420537291096
Final  0 0.31555180273416067
Final  0 0.0
Final  0 0.0
Final  0 0.0
Final  0 0.0
Final  0 0.31555180273416067
Final  0 0.31555180273416067
Final  0 0.315551802734160

/home/roshan/anaconda3/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


,bank,bike,bike loan,business,business services,consumer,consumer durable,durable,durable loan,finance,...,loan,mobile,mobile loan,mutual,mutual fund,personal,personal loan,services,wheeler,wheeler loan
0,0.315552,0.0,0.0,0.315552,0.0,0.0,0.0,0.0,0.0,0.275534,...,0.167106,0.315552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
model = MultinomialNB(alpha = 1)
model.fit(X,y)
model.predict(X_res)

NameError: name 'MultinomialNB' is not defined

In [67]:
modelknn = KNeighborsClassifier(n_neighbors=(k+1))
modelknn.fit(X_train,Y_train)
modelknn.predict(X_res)
   

array(['Bank'], dtype=object)